In [24]:
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
import os
from pydantic import BaseModel
from typing import Optional, Union, Type
from enum import Enum

In [ ]:
load_dotenv(override=True)


In [17]:
# Open AI LLMs call
# OpenAI Model Enums
class OpenAIModel(str, Enum):
    """Available OpenAI models"""
    GPT_4 = "gpt-4"
    GPT_4_TURBO = "gpt-4-turbo"
    GPT_4O = "gpt-4o"
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_3_5_TURBO = "gpt-3.5-turbo"
    O1_PREVIEW = "o1-preview"
    O1_MINI = "o1-mini"

def gpt_calls(
    prompt: str, 
    system_prompt: str = "",
    model: OpenAIModel = OpenAIModel.GPT_4O_MINI,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None
) -> Union[str, BaseModel]:
    """
    Simple OpenAI API caller
    
    Args:
        prompt: The prompt to send
        model: Model to use (from OpenAIModel enum)
        system_prompt: System message to set behavior/context (optional)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature, temperature controls the randomness or creativity of the LLM's responses.
    
    Returns:
        String message if no response_format, or Pydantic object if response_format provided
    """
    openai = OpenAI()
    # Build messages array
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    call_params = {
            "model": model.value,
            "messages": messages,
            "temperature": temperature
        }

    if max_tokens:
            call_params["max_tokens"] = max_tokens
    # If response_format is provided, use structured output
    if response_format:
        completion = openai.chat.completions.parse(
            **call_params,
            response_format = response_format
        )
        return completion.completion.choices[0].message.parsed

    # Otherwise return plain text
    else:
        completion = openai.chat.completions.create(**call_params)
        return completion.choices[0].message.content

In [18]:
gpt_calls("Hi there")

'Hello! How can I assist you today?'

In [19]:

class GeminiModel(str, Enum):
    """Available Gemini models"""
    GEMINI_2_0_FLASH = "gemini-2.0-flash"
    GEMINI_1_5_FLASH = "gemini-1.5-flash"
    GEMINI_1_5_FLASH_8B = "gemini-1.5-flash-8b"
    GEMINI_1_5_PRO = "gemini-1.5-pro"
    GEMINI_EXP_1206 = "gemini-exp-1206"

def gemini_calls(
    prompt: str, 
    system_prompt: str = "",
    model: GeminiModel = GeminiModel.GEMINI_2_0_FLASH,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None
) -> Union[str, BaseModel]:
    """
    Simple Gemini API caller
    
    Args:
        prompt: The prompt to send
        model: Model to use (from OpenAIModel enum)
        system_prompt: System message to set behavior/context (optional)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature, temperature controls the randomness or creativity of the LLM's responses.
    
    Returns:
        String message if no response_format, or Pydantic object if response_format provided
    """
    openai = OpenAI(
        api_key=os.getenv('GOOGLE_API_KEY'),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    call_params = {
            "model": model.value,
            "messages": messages,
            "temperature": temperature
        }

    if max_tokens:
            call_params["max_tokens"] = max_tokens
    # If response_format is provided, use structured output
    if response_format:
        completion = openai.chat.completions.parse(
            **call_params,
            response_format = response_format
        )
        return completion.completion.choices[0].message.parsed

    # Otherwise return plain text
    else:
        completion = openai.chat.completions.create(**call_params)
        return completion.choices[0].message.content

In [20]:
gemini_calls("Hi there!")

'Hi! How can I help you today?\n'

In [40]:
"""
Unified LLM Caller for OpenAI GPT and Google Gemini, Claude
"""

import os
from openai import OpenAI
from anthropic import Anthropic
from pydantic import BaseModel
from typing import Optional, Union, Type, List, Dict
from enum import Enum


class OpenAIModel(str, Enum):
    """Available OpenAI models"""
    GPT_4 = "gpt-4"
    GPT_4_TURBO = "gpt-4-turbo"
    GPT_4O = "gpt-4o"
    GPT_4O_MINI = "gpt-4o-mini"
    GPT_3_5_TURBO = "gpt-3.5-turbo"
    O1_PREVIEW = "o1-preview"
    O1_MINI = "o1-mini"


class GeminiModel(str, Enum):
    """Available Gemini models"""
    GEMINI_2_0_FLASH = "gemini-2.0-flash"
    GEMINI_1_5_FLASH = "gemini-1.5-flash"
    GEMINI_1_5_FLASH_8B = "gemini-1.5-flash-8b"
    GEMINI_1_5_PRO = "gemini-1.5-pro"
    GEMINI_EXP_1206 = "gemini-exp-1206"

class ClaudeModel(str, Enum):
    """Available Claude models"""
    CLAUDE_3_5_SONNET = "claude-3-5-sonnet-20241022"
    CLAUDE_3_5_HAIKU = "claude-3-5-haiku-20241022"
    CLAUDE_3_OPUS = "claude-3-opus-20240229"
    CLAUDE_3_SONNET = "claude-3-sonnet-20240229"
    CLAUDE_3_HAIKU = "claude-3-haiku-20240307"


def _build_messages(prompt: str, system_prompt: Optional[str] = None) -> List[Dict[str, str]]:
    """Build messages array for API call"""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    return messages


def _call_llm(
    client: OpenAI,
    prompt: str,
    model: str,
    system_prompt: Optional[str] = None,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None
) -> Union[str, BaseModel]:
    """
    Internal function to call LLM API
    
    Args:
        client: OpenAI client instance
        prompt: The prompt to send
        model: Model string to use
        system_prompt: System message (optional)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature (0.0-2.0)
        max_tokens: Maximum tokens in response (optional)
    
    Returns:
        String or Pydantic object
    """
    messages = _build_messages(prompt, system_prompt)
    
    call_params = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }
    
    if max_tokens:
        call_params["max_tokens"] = max_tokens
    
    if response_format:
        completion = client.beta.chat.completions.parse(
            **call_params,
            response_format=response_format
        )
        return completion.choices[0].message.parsed
    else:
        completion = client.chat.completions.create(**call_params)
        return completion.choices[0].message.content


def gpt_call(
    prompt: str,
    system_prompt: Optional[str] = None,
    model: OpenAIModel = OpenAIModel.GPT_4O_MINI,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None,
    api_key: Optional[str] = None
) -> Union[str, BaseModel]:
    """
    Call OpenAI GPT API
    
    Args:
        prompt: The prompt to send
        system_prompt: System message to set behavior/context (optional)
        model: Model to use (default: GPT-4o-mini)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature, controls randomness/creativity (0.0-2.0)
        max_tokens: Maximum tokens in response (optional)
        api_key: OpenAI API key (optional, uses OPENAI_API_KEY env var if not provided)
    
    Returns:
        String message if no response_format, or Pydantic object if provided
    """
    client = OpenAI(api_key=api_key) if api_key else OpenAI()
    
    return _call_llm(
        client=client,
        prompt=prompt,
        model=model.value,
        system_prompt=system_prompt,
        response_format=response_format,
        temperature=temperature,
        max_tokens=max_tokens
    )


def gemini_call(
    prompt: str,
    system_prompt: Optional[str] = None,
    model: GeminiModel = GeminiModel.GEMINI_2_0_FLASH,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None,
    api_key: Optional[str] = None
) -> Union[str, BaseModel]:
    """
    Call Google Gemini API
    
    Args:
        prompt: The prompt to send
        system_prompt: System message to set behavior/context (optional)
        model: Model to use (default: Gemini 2.0 Flash)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature, controls randomness/creativity (0.0-2.0)
        max_tokens: Maximum tokens in response (optional)
        api_key: Google AI API key (optional, uses GOOGLE_API_KEY env var if not provided)
    
    Returns:
        String message if no response_format, or Pydantic object if provided
    """
    api_key = api_key or os.getenv('GOOGLE_API_KEY')
    
    client = OpenAI(
        api_key=api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    
    return _call_llm(
        client=client,
        prompt=prompt,
        model=model.value,
        system_prompt=system_prompt,
        response_format=response_format,
        temperature=temperature,
        max_tokens=max_tokens
    )


def claude_call(
    prompt: str,
    system_prompt: Optional[str] = None,
    model: ClaudeModel = ClaudeModel.CLAUDE_3_HAIKU,
    response_format: Optional[Type[BaseModel]] = None,
    temperature: float = 0.7,
    max_tokens: Optional[int] = 1024,
    api_key: Optional[str] = None
) -> Union[str, BaseModel]:
    """
    Call Anthropic Claude API
    
    Args:
        prompt: The prompt to send
        system_prompt: System message to set behavior/context (optional)
        model: Model to use (default: Claude 3.5 Sonnet)
        response_format: Pydantic model class for structured output (optional)
        temperature: Sampling temperature, controls randomness/creativity (0.0-1.0)
        max_tokens: Maximum tokens in response (default: 1024)
        api_key: Anthropic API key (optional, uses ANTHROPIC_API_KEY env var if not provided)
    
    Returns:
        String message if no response_format, or Pydantic object if provided
    """
    api_key = api_key or os.getenv('ANTHROPIC_API_KEY')
    client = Anthropic(api_key=api_key)
    
    # Build messages
    messages = _build_messages(prompt, system_prompt=None)  # Claude handles system separately
    
    # Prepare call parameters
    call_params = {
        "model": model.value,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    
    # Add system prompt if provided (Claude uses separate system parameter)
    if system_prompt:
        call_params["system"] = system_prompt
    print(call_params)
    # Call API
    if response_format:
        # Structured output with Pydantic
        completion = client.messages.create(
            **call_params,
            response_format=response_format
        )
        # Parse the response based on Pydantic model
        return response_format.model_validate_json(completion.content[0].text)
    else:
        # Plain text output
        completion = client.messages.create(**call_params)
        return completion.content[0].text


In [44]:
claude_call("Tell me a joke about programmers.", "You are a formal and polite assistant. Always reply in Shakespearean English.")

{'model': 'claude-3-haiku-20240307', 'messages': [{'role': 'user', 'content': 'Tell me a joke about programmers.'}], 'temperature': 0.7, 'max_tokens': 1024, 'system': 'You are a formal and polite assistant. Always reply in Shakespearean English.'}


"Wherefore doth the programmer oft complain,\nOf bugs that vex their code, causing them pain?\nFor they toil with keyboard and screen all day,\nHoping their programs will work, come what may.\n\nYet, alas, the computer doth oft refuse,\nTo obey their commands, and their efforts abuse.\nThey scratch their heads, and ponder with might,\nSeeking the elusive solution, through the endless night.\n\nBut fear not, for in their frustration, they find,\nA moment of levity, to ease their troubled mind.\nFor what is a programmer, if not a jester of sorts,\nWhose jokes and puns, the tech world ever courts?\n\nSo let us laugh, and embrace our inner nerd,\nFor the programmer's life, is truly absurd.\nAnd may their code be ever bug-free and bright,\nAs they continue their quest, for programming's delight."